In [3]:
from simulation_utils import data_generator1
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y, task_type, meta_info, get_metric = data_generator1(datanum=1000, random_state=0)

tr_x, val_x, tr_y, val_y = train_test_split(
    train_x, train_y,
    test_size=0.2,
    random_state=0
)

In [4]:
from neural_additive_models import graph_builder

learning_rate = 0.0001
batch_size = 256
l2_regularization = 1e-5
output_regularization = 0.0
dropout = 0.0
decay_rate = 0.0
feature_dropout = 0.0
num_basis_functions = 128
units_multiplier = 1.0
shallow = False
activation = 'relu'
use_dnn = False
regression = True

training_epochs = 5 # 5000
early_stopping_epochs = 50


In [5]:
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_eager_execution()

tr_y_nam  = tr_y.reshape(-1).astype(np.float32)
val_y_nam = val_y.reshape(-1).astype(np.float32)
test_y_nam = test_y.reshape(-1).astype(np.float32)


graph, metrics = graph_builder.build_graph(
    x_train=tr_x,
    y_train=tr_y_nam,
    x_test=val_x,                   # NAM test = our VAL set
    y_test=val_y_nam,
    activation=activation,
    learning_rate=learning_rate,
    batch_size=batch_size,
    shallow=shallow,
    output_regularization=output_regularization,
    l2_regularization=l2_regularization,
    dropout=dropout,
    num_basis_functions=num_basis_functions,
    units_multiplier=units_multiplier,
    decay_rate=decay_rate,
    feature_dropout=feature_dropout,
    regression=regression,
    use_dnn=use_dnn,
    trainable=True,
    name_scope="model_0"
)


Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.
Model: "nam"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_nn (FeatureNN)      multiple                  10624     
                                                                 
 feature_nn_1 (FeatureNN)    multiple                  10624     
                                                            

In [6]:
train_op = graph['train_op']
initializer = [graph['iterator_initializer'], graph['running_vars_initializer']]

best_val = np.inf
patience = 0

with tf.train.MonitoredSession() as sess:
    sess.run(initializer)

    for epoch in range(training_epochs):
        # Run one epoch using GAMI-Net's training split!
        for _ in range(tr_x.shape[0] // batch_size):
            sess.run(train_op)

        # Compute validation RMSE (same val_x as GAMI-Net)
        val_rmse = metrics['test'](sess)

        if val_rmse < best_val:
            best_val = val_rmse
            patience = 0
        else:
            patience += 1

        print(f"Epoch {epoch:3d}  |  val RMSE = {val_rmse:.5f}")

        if patience >= early_stopping_epochs:
            print("Early stopping triggered.")
            break

    trained_sess = sess


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch   0  |  val RMSE = 0.31247
Epoch   1  |  val RMSE = 0.26924
Epoch   2  |  val RMSE = 0.32241
Epoch   3  |  val RMSE = 0.29847
Epoch   4  |  val RMSE = 0.26064


In [7]:
tf.reset_default_graph()

graph_test, metrics_test = graph_builder.build_graph(
    x_train=tr_x,
    y_train=tr_y_nam,
    x_test=test_x,            # ← REAL test set
    y_test=test_y_nam,
    activation=activation,
    learning_rate=learning_rate,
    batch_size=batch_size,
    shallow=shallow,
    output_regularization=0.0,     # OK for testing
    l2_regularization=0.0,         # MUST be 0 for testing to avoid L2 crash
    dropout=dropout,
    num_basis_functions=num_basis_functions,
    units_multiplier=units_multiplier,
    decay_rate=decay_rate,
    feature_dropout=feature_dropout,
    regression=regression,
    use_dnn=use_dnn,
    trainable=True,                # keep True to avoid optimizer crash
    name_scope="nam_test"
)

with tf.train.MonitoredSession() as sess:
    sess.run([
        graph_test["iterator_initializer"],      # loads tr_x/tr_y
        graph_test["running_vars_initializer"]
    ])

    nam_test_rmse = metrics_test["test"](sess)   # computes RMSE on test_x/test_y

print("NAM Test RMSE:", nam_test_rmse)



Model: "nam"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_nn (FeatureNN)      multiple                  10624     
                                                                 
 feature_nn_1 (FeatureNN)    multiple                  10624     
                                                                 
 feature_nn_2 (FeatureNN)    multiple                  10624     
                                                                 
 feature_nn_3 (FeatureNN)    multiple                  10624     
                                                                 
 feature_nn_4 (FeatureNN)    multiple                  10624     
                                                                 
 feature_nn_5 (FeatureNN)    multiple                  10624     
                                                                 
 feature_nn_6 (FeatureNN)    multiple                  10624   